In [ ]:
%pip install python-dotenv google-generativeai line-bot-sdk flask google-cloud-storage

In [1]:
import os
import traceback
import json
from io import BytesIO
from PIL import Image
from flask import Flask, request, abort

# 新增：引入 dotenv
from dotenv import load_dotenv

# 引入 Gemini 套件
import google.generativeai as genai

# 引入 LINE Bot V3 套件
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent,
    LocationMessageContent
)

# 假設 cwa 模組 (若無此檔案可先註解掉)
try:
    import cwa
except ImportError:
    cwa = None

from google.cloud import storage

# --- 1. 初始化環境變數 ---
# 這行會自動尋找並讀取 .env 檔案中的變數到 os.environ 中
load_dotenv()

# 從環境變數讀取金鑰
CHANNEL_ACCESS_TOKEN = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
CHANNEL_SECRET = os.getenv('LINE_CHANNEL_SECRET')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
CWA_KEY = os.getenv('CWA_KEY')
GCP_KEY_PATH = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# 檢查必要的 Key 是否存在 (除錯用)
if not CHANNEL_ACCESS_TOKEN or not GEMINI_API_KEY:
    print("警告：未讀取到 LINE 或 Gemini 的金鑰，請檢查 .env 檔案")

# --- 2. 初始化服務 ---

# 設定 Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash')

# 設定 Google Cloud Storage
# 注意：Google 官方套件會自動尋找 'GOOGLE_APPLICATION_CREDENTIALS' 環境變數
# 所以通常不需要額外指定，但為了確保程式碼運作，我們維持顯式呼叫
try:
    if GCP_KEY_PATH and os.path.exists(GCP_KEY_PATH):
        storage_client = storage.Client.from_service_account_json(GCP_KEY_PATH)
        bucket = storage_client.bucket('leofrstproject')
    else:
        print("找不到 GCP Key 檔案，略過 Storage 設定")
        bucket = None
except Exception as e:
    print(f"GCP Storage 設定錯誤: {e}")
    bucket = None

app = Flask(__name__)

configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# --- 3. 輔助函式：清理 JSON ---
def clean_json_text(text):
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]
    if text.startswith("```"):
        text = text[3:]
    if text.endswith("```"):
        text = text[:-3]
    return text.strip()

# --- 4. Webhook 入口 ---
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)
    return 'OK'

# --- 5. 文字訊息處理 (修正版) ---
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    ask = event.message.text
    # 1. 將輸入轉為小寫，解決 "HI" vs "hi" 的問題
    ask_lower = ask.lower()
    
    ask_map = {
        'hello': '我很好', 
        'hi': '您哪位',
        '你好': '你好呀！傳張寵物照片給我看看？' # 順便多加一個中文問候
    }
    
    # 2. 查表 (使用小寫後的字串)
    ans = ask_map.get(ask_lower)
    
    # 3. 若不是問候語，且有氣象模組，嘗試查氣象
    if not ans and cwa:
        try:
            # 嘗試查詢氣象 (這裡用原始文字 ask，因為地名通常是中文)
            weather_data = cwa.cwa2(ask, CWA_KEY)
            
            # 如果有查到資料，就轉換成字串
            if weather_data:
                ans = cwa.tostr(weather_data, '\n')
            else:
                # 查不到氣象，視為一般閒聊，不顯示錯誤訊息
                ans = None 
        except Exception as e:
            # 發生錯誤時印出 Log，但不回傳錯誤給使用者
            print(f"氣象查詢錯誤: {e}")
            ans = None

    # 4. 如果問候語和氣象都沒結果，回覆預設引導訊息
    if not ans:
        ans = "我聽不懂你在說什麼～試試傳一張寵物照片給我！🐶🐱"

    # 5. 回傳訊息
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

# --- 6. 地點訊息處理 ---
@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    if not cwa:
        return
        
    site = (event.message.latitude, event.message.longitude)
    try:
        ans = cwa.cwa2(site, CWA_KEY) # 使用環境變數
        ans = cwa.tostr(ans, '\n') or '無此站'
    except:
        ans = "無法查詢該地點氣象"

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

# --- 7. 圖片訊息處理 (Debug 版) ---
@handler.add(MessageEvent, message=ImageMessageContent)
def handle_content_message(event):
    print("-----------------------------------------")
    print(f"1. 收到圖片訊息！ID: {event.message.id}")
    
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        image_bytes = message_content
        image = Image.open(BytesIO(image_bytes))
        print("2. 圖片已下載並轉換成功")

        # 上傳 GCP (維持原樣，略過不印)
        if bucket:
            try:
                blob_name = f'{event.source.user_id}_{event.message.id}'
                blob = bucket.blob(blob_name)
                blob.upload_from_string(image_bytes, content_type='image/jpeg')
            except Exception as e:
                print(f"GCP Upload Error: {e}")

        # Gemini 辨識
        try:
            print("3. 正準備呼叫 Gemini API...")
            # 簡化版 Prompt
            prompt = """
            請分析這張圖片。
            第一步：判斷圖片主體是否為「真實的動物寵物」（如貓、狗、兔、倉鼠、鳥等）。
            第二步：回傳 JSON 格式結果。
            若「不是寵物」，回傳： {"is_pet": false}
            若「是寵物」，回傳（繁體中文）：
            {"is_pet": true, "species": "物種", "breed": "品種", "colors": ["顏色"], "mood": "情緒", "features": "特徵", "care_tips": "建議"}
            只回傳 JSON。
            """

            response = model.generate_content([prompt, image])
            print("4. Gemini 回傳成功！內容如下：")
            print(response.text)  # 印出 AI 回傳的原始文字，確認有沒有東西
            
            # 清理與解析
            json_str = clean_json_text(response.text)
            import json
            data = json.loads(json_str)

            if not data.get("is_pet"):
                reply_text = "這不是毛小孩相片 🐶🐱"
                print("5. 判斷結果：非寵物")
            else:
                reply_text = (
                    f"這是一隻可愛的 {data.get('breed', '毛小孩')} ({data.get('species')})！\n"
                    f"🎨 毛色：{', '.join(data.get('colors', []))}\n"
                    f"😺 心情：{data.get('mood')}\n"
                    f"📝 特徵：{data.get('features')}\n"
                    f"💡 照顧建議：{data.get('care_tips')}"
                )
                print("5. 判斷結果：是寵物，已生成回覆文字")

        except Exception as e:
            traceback.print_exc()
            reply_text = "AI 辨識發生錯誤，請稍後再試。"
            print("!!! 發生錯誤，已印出 Traceback !!!")

        # 回傳給 LINE
        try:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            print("6. 訊息已發送回 LINE 伺服器")
        except Exception as e:
            print(f"!!! LINE 回傳失敗: {e}")
            print("請檢查 .env 中的 LINE_CHANNEL_ACCESS_TOKEN 是否正確")


if __name__ == "__main__":
    app.run(port=8080)


c:\Users\lyhua\miniconda3\envs\pyla\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lyhua\AppData\Local\Temp\ipykernel_1624\1184926970.py:12: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


sitemaps built


127.0.0.1 - - [28/Dec/2025 16:55:56] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2025 16:57:15] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2025 16:57:22] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2025 16:57:33] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2025 16:58:18] "POST /callback HTTP/1.1" 200 -


In [2]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# 1. 載入環境變數
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("❌ 錯誤：找不到 API Key，請檢查 .env 檔案")
else:
    print(f"🔑 API Key 前五碼: {api_key[:5]}...") # 確認有讀到 Key
    
    # 2. 設定 API
    genai.configure(api_key=api_key)

    print("\n📡 正在查詢 Google 伺服器可用模型列表...")
    try:
        found_any = False
        for m in genai.list_models():
            # 只列出支援 'generateContent' (文字/圖片生成) 的模型
            if 'generateContent' in m.supported_generation_methods:
                print(f"✅ 可用模型: {m.name}")
                found_any = True
        
        if not found_any:
            print("⚠️ 警告：連線成功，但沒有找到任何支援生成的模型。可能是 Key 的權限問題。")
            
    except Exception as e:
        print(f"❌ 查詢失敗: {e}")

🔑 API Key 前五碼: AIzaS...

📡 正在查詢 Google 伺服器可用模型列表...
✅ 可用模型: models/gemini-2.5-flash
✅ 可用模型: models/gemini-2.5-pro
✅ 可用模型: models/gemini-2.0-flash-exp
✅ 可用模型: models/gemini-2.0-flash
✅ 可用模型: models/gemini-2.0-flash-001
✅ 可用模型: models/gemini-2.0-flash-exp-image-generation
✅ 可用模型: models/gemini-2.0-flash-lite-001
✅ 可用模型: models/gemini-2.0-flash-lite
✅ 可用模型: models/gemini-2.0-flash-lite-preview-02-05
✅ 可用模型: models/gemini-2.0-flash-lite-preview
✅ 可用模型: models/gemini-exp-1206
✅ 可用模型: models/gemini-2.5-flash-preview-tts
✅ 可用模型: models/gemini-2.5-pro-preview-tts
✅ 可用模型: models/gemma-3-1b-it
✅ 可用模型: models/gemma-3-4b-it
✅ 可用模型: models/gemma-3-12b-it
✅ 可用模型: models/gemma-3-27b-it
✅ 可用模型: models/gemma-3n-e4b-it
✅ 可用模型: models/gemma-3n-e2b-it
✅ 可用模型: models/gemini-flash-latest
✅ 可用模型: models/gemini-flash-lite-latest
✅ 可用模型: models/gemini-pro-latest
✅ 可用模型: models/gemini-2.5-flash-lite
✅ 可用模型: models/gemini-2.5-flash-image-preview
✅ 可用模型: models/gemini-2.5-flash-image
✅ 可用模型: models/gemini-2.5-fla